## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/DSL/22-2/text mining

/content/drive/MyDrive/DSL/22-2/text mining


In [ ]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu

In [ ]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
os.getcwd()

'/content/drive/MyDrive/DSL/22-2/text mining'

In [ ]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [ ]:
corpus[:50]

['소두증을 유발한다고 알려진 지카바이러스가 남성의 생식능력도 저하시킬 수 있다는 동물 실험 결과가 나왔다',
 '1일 국제학술지 네이처에 따르면 미 워싱턴대 의대 연구진은 수컷 쥐가 지카바이러스에 감염되면 고환 크기가 눈에 띄게 작아지며 정자 수가 감소하고, 성 호르몬인 테스토스테론의 양도 줄어든다는 연구 결과를 발표했다',
 '지금까지 지카바이러스는 태아에게 소두증을 일으킨다고 알려져 태아와 여성의 생식기관 감염에 초점을 맞춘 연구들이 많았다',
 '소두증은 태아의 뇌가 다 자라지 않아 머리가 비정상적으로 작아지는 질환이다',
 '최근에는 브라질 등 남미 뿐 아니라 미국, 동남아시아에서도 환자가 발생하고 있다',
 '워싱턴대 연구진은 지카바이러스가 남성의 생식기관에 미치는 영향에 초점을 맞춰 연구를 진행했다',
 '연구진은 우선 수컷 쥐에게 지카바이러스를 감염시켰다',
 '1주가 지나자 생식기관인 고환에서도 바이러스가 발견됐다',
 '2주 뒤에는 수컷 쥐의 고환 크기가 현격하게 줄고 무게도 감소했다',
 '일반 쥐의 고환 무게는 75mg 이상이지만, 바이러스에 감염된 쥐의 경우 50mg도 되지 않았다',
 '3주 뒤 쥐의 고환 크기는 더욱 감소했으며 무게는 25mg 미만으로 줄어들었다',
 '연구진은 고환을 구성하는 세포가 죽었고, 고환 내부의 구조도 손상된 것을 확인했다',
 '수컷의 핵심 생식기관인 고환이 지카바이러스의 공격으로 점차 기능을 잃어간 것이다',
 '고환은 생식세포인 정자와 성호르몬인 테스토스테론을 만드는 기관이다',
 '지카바이러스에 감염된 쥐는 고환의 크기가 작을 뿐 아니라 정자 수와 성호르몬 수치도 정상에 비해 적었다',
 '정자의 운동성도 현저히 감소했다',
 '오명돈 서울대병원 감염내과 교수는 “동물 모델을 이용해 지카바이러스가 정모세포(정자로 성장하는 세포), 정세관(정자가 나오는 작은 튜브) 세포 손상을 일으킨다는 것과 남성호르몬과 정자수, 고환크기, 가임력을 모두 감소시킨다는 것을 구체적으로 밝혔다”며 “다만 쥐 실험이므로 사

In [ ]:
len(corpus)

1020

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

stop words를 미리 지정하고 pos tagging을 사용하여 단어의 품사가 stop words에 속하면 그것은 제외하고 나머지는 token화 시키는 방향으로 작업하였다. noun으로 잘못 인식하는 경우가 많아서 개선이 필요하다

In [ ]:
okt = Okt()

In [ ]:
ex1 = '데이터 사이언스 랩은 텍스트 마이닝에 대해서도 공부할 수 있는 좋은 학회입니다.'

In [ ]:
okt.pos(ex1, stem=True, norm=True)

[('데이터', 'Noun'),
 ('사이언스', 'Noun'),
 ('랩', 'Noun'),
 ('은', 'Josa'),
 ('텍스트', 'Noun'),
 ('마', 'Noun'),
 ('이닝', 'Noun'),
 ('에', 'Josa'),
 ('대해', 'Noun'),
 ('서도', 'Noun'),
 ('공부', 'Noun'),
 ('하다', 'Verb'),
 ('수', 'Noun'),
 ('있다', 'Adjective'),
 ('좋다', 'Adjective'),
 ('학회', 'Noun'),
 ('이다', 'Adjective'),
 ('.', 'Punctuation')]

In [ ]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
    stop = ['Josa','Suffix','Punctuation','Foreign', 'Number']
    token = []
    for i, j in okt.pos(sentence, stem=True, norm=True):
        if j in stop:
            continue
        token.append(i)
    return token

In [ ]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [ ]:
corpus_tokenized[:5]

[['소',
  '두',
  '증',
  '유발',
  '하다',
  '알려지다',
  '지',
  '카',
  '바이러스',
  '남성',
  '생식능력',
  '저하',
  '시키다',
  '수',
  '있다',
  '동물',
  '실험',
  '결과',
  '나오다'],
 ['국제',
  '학술지',
  '네이처',
  '따르다',
  '밉다',
  '워싱턴',
  '의대',
  '연',
  '구진',
  '수컷',
  '쥐',
  '지',
  '카',
  '바이러스',
  '감염',
  '되다',
  '고환',
  '크기',
  '눈',
  '띄다',
  '작아지다',
  '정자',
  '수가',
  '감소',
  '성',
  '호르몬',
  '테스토스테론',
  '양도',
  '줄어들다',
  '연구',
  '결과',
  '발표',
  '하다'],
 ['지금',
  '지',
  '카',
  '바이러스',
  '태아',
  '소',
  '두',
  '증',
  '일으키다',
  '알려지다',
  '태아',
  '여성',
  '생식기관',
  '감염',
  '초점',
  '맞추다',
  '연구',
  '많다'],
 ['소', '두', '증', '태아', '뇌', '다', '자라다', '않다', '머리', '비정상', '작아지다', '질환'],
 ['최근', '브라질', '등', '남미', '뿐', '아니다', '미국', '동남아시아', '환자', '발생', '있다']]

In [ ]:
len(corpus_tokenized)

1020

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

가지고 있는 데이터가 크지 않고, 중요한 단어일지라도 출현 빈도가 적을 것이라고 생각했기 때문에 min_count를 0으로 두었으며, window 또한 줄였다. 

In [ ]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(min_count = 0, window=3, size=300)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3808


In [ ]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 5)

(74945, 84210)

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('바이러스')

[('받다', 0.47161024808883667),
 ('가다', 0.47144174575805664),
 ('기술', 0.4674583971500397),
 ('만들다', 0.4592271149158478),
 ('교수', 0.45876842737197876),
 ('물질', 0.45611098408699036),
 ('의', 0.4532732367515564),
 ('하다', 0.4502468705177307),
 ('연', 0.4497283399105072),
 ('이', 0.4490870237350464)]

In [ ]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size = 200, min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 3808


In [ ]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31436


In [ ]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 4)

(66618, 67368)

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('바이러스')

[('박테리아', 0.7474043965339661),
 ('병원체', 0.7302976250648499),
 ('감염', 0.6540313363075256),
 ('종양', 0.6508430242538452),
 ('인플루엔자', 0.6419334411621094),
 ('백신', 0.636033296585083),
 ('항생제', 0.6296895146369934),
 ('기생충', 0.6180971264839172),
 ('항체', 0.6072112917900085),
 ('백혈구', 0.6018475294113159)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

사전학습을 사용한 모델은 vocabulary까지 불러옴으로 비슷한 단어로 학습시키기 수월했을 것이기 때문에 성능이 좋았다.
사전학습없이 사용한 모델은 epoch을 조금만 늘려도 유사도가 0.9 이상으로 급속히 늘어나며, 유사한 단어를 찾기도 쉽지 않았다.


# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [ ]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3808

In [ ]:
unique_text[:3]

['밉다', '사랑', '화석']

In [ ]:
# embedding 결과 가져오기
word, vector_pre, vector_nopre = [], [], []
for i in unique_text:
    word.append(i)
    vector_nopre.append(model_no_pretrained.wv[i])
    vector_pre.append(model_pretrained.wv[i])

In [ ]:
pca = PCA(random_state=27).fit_transform(vector_pre)
tsne = TSNE(random_state=17).fit_transform(vector_pre)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [ ]:
pca.shape

(3808, 200)

In [ ]:
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in range(pca.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [ ]:
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(tsne.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [ ]:
pca_nopre = PCA(random_state=27).fit_transform(vector_nopre)
tsne_nopre = TSNE(random_state=17).fit_transform(vector_nopre)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [ ]:
data = go.Scatter(
    x=pca_nopre[:,0],
    y=pca_nopre[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

In [ ]:
data = go.Scatter(
    x=tsne_nopre[:,0],
    y=tsne_nopre[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [ ]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 17, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector_pre)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [ ]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 17, perplexity = 30, learning_rate = 0.05, init = 'random', n_iter = 500).fit_transform(vector_pre)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in word],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




perplexity와 niter를 줄이니 중앙으로 모이는 경향 있었다

pretrained된 vector를 사용해서 인지 비슷한 단어끼리 뭉쳐있는 듯 했다

# 수고하셨습니다!